In [ ]:
from pyspark.sql import functions as F

CATALOGO = "spotify_analytics"
SCHEMA = "gold"
TABELA = "ft_track_popularity"
FULL_NAME = f"{CATALOGO}.{SCHEMA}.{TABELA}"

### Configuração

Define catálogo, schema e nome da tabela fato principal (camada Gold - Star Schema).

### DDL - Fato Track Popularity

Cria tabela fato principal com métricas de popularidade de tracks:

**Estrutura:**
- **pk_fato_track** (PK - STRING): Surrogate key (hash MD5)
- **Foreign Keys**: cd_artist, cd_track, cd_tempo
- **Métricas**: val_popularity, val_duration_ms
- **Flags de Negócio**:
  - fl_explicit: Conteúdo explícito (0/1)
  - fl_alta_popularidade: Popularidade > 70 (0/1)
  - fl_mega_hit: Popularidade > 90 (0/1)
  - fl_track_curta: Duração < 3 minutos (0/1)

**Decisões Arquiteturais:**

1. **Por que MD5 hash como PK?**
   - Idempotência: Mesmo input → mesma key
   - Distribuição uniforme em Spark
   - Evita conflitos de autoincrement em ambiente distribuído
   - Facilita MERGE (upsert)

2. **Por que flags de negócio (fl_)?**
   - Pré-processamento: Evita cálculos repetidos em BI
   - Self-service: Analistas podem filtrar sem conhecer regras
   - Performance: Filtros em SMALLINT (0/1) são mais rápidos que expressões
   - Documentação: Regras de negócio ficam explícitas no código ETL

3. **Por que métricas com prefixo val_?**
   - Padrão de nomenclatura que diferencia dimensões de métricas
   - Facilita identificação em ferramentas de BI
   - Seguindo convenção de mercado (val = value)

**Importante**: Fato central do Star Schema para análises de popularidade e tendências musicais.

In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {FULL_NAME} (
    pk_fato_track STRING NOT NULL COMMENT 'PK: Hash MD5 único da linha.',
    
    cd_artist BIGINT COMMENT 'FK: Liga com dm_artist.',
    cd_track BIGINT COMMENT 'FK: Liga com dm_track.',
    cd_tempo INT COMMENT 'FK: Liga com dm_tempo.',
    
    val_popularity INT COMMENT 'Métrica: Score de popularidade (0-100).',
    val_duration_ms BIGINT COMMENT 'Métrica: Duração da track em milissegundos.',
    
    fl_explicit SMALLINT COMMENT 'Flag (0/1): Track possui conteúdo explícito.',
    fl_alta_popularidade SMALLINT COMMENT 'Flag (0/1): Popularidade > 70.',
    fl_mega_hit SMALLINT COMMENT 'Flag (0/1): Popularidade > 90.',
    fl_track_curta SMALLINT COMMENT 'Flag (0/1): Duração < 180000ms (3 minutos).',
    
    CONSTRAINT pk_ft_track_popularity PRIMARY KEY (pk_fato_track)
)
COMMENT 'Fato principal: métricas de popularidade e flags de negócio para analytics.';
""")

print(f"✅ Tabela {FULL_NAME} criada com sucesso!")